# Robust edge detection through active visual sampling via fixational eye movements - A run through the model

## Overview
- Model overview
- Imports
- (A) Example input
- (B) Active sampling / Applying drift
- (C) Spatiotemporal filtering
- (D) Temporal integration
- (E) Within-scale normalization
- (F) Across-scale integration / Sum
- (G) Evaluation

## Model overview
![model](model_overview.png "Model")

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import os
import cv2
from matplotlib import animation
from IPython.display import HTML

# Read functions from parent folder:
sys.path.append('../simulations')
import parameters as params
from functions import create_drift, apply_drift, octave_intervals, create_dog, create_tfilt, \
remove_borders, create_white_stimulus, create_noisemask, create_white_gt, quantify_edges, thicken_edges

In [ ]:
ffmpeg_path = r'C:\Program Files\FFMPEG\ffmpeg-5.1.2-essentials_build\bin\ffmpeg.exe' # replace with your own path

In [ ]:
def show_video(vid_arr, cmap='gray'):
    """Helper function to display a video

    Parameters
    ----------
    vid_arr
        3D video array with dimensions (x, y, t)
    cmap
        Colormap used for plotting (default: grayscale)

    """
    %matplotlib inline
    fig = plt.figure(figsize=(8,8))
    im = plt.imshow(vid_arr[:,:,0], cmap=cmap)
    plt.axis('off')
    plt.close()

    def init():
        im.set_data(vid_arr[:,:,0])

    def animate(i):
        im.set_data(vid_arr[:,:,i])
        return im

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=vid_arr.shape[2], interval=50)
    plt.rcParams['animation.ffmpeg_path'] = ffmpeg_path
    return anim


def show_filter_videos(vid_arr, cmap='gray'):
    """Helper function to display videos of the filter outputs of our model
    at all five spatial scales simultanouesly

    Parameters
    ----------
    vid_arr
        4D video array with dimensions (x, y, t, filter_scale).
        The last dimensions needs to have a size of 5.
    cmap
        Colormap used for plotting (default: grayscale)

    """
    %matplotlib inline
    n_filters = vid_arr.shape[-1]
    fig, [ax1, ax2, ax3, ax4, ax5] = plt.subplots(1, n_filters, figsize=(12,4), sharex=True, sharey=True)
    fig.subplots_adjust(wspace=0.01)
    
    # initialize
    im1 = ax1.imshow(vid_arr[:,:,0,0], cmap=cmap, animated=True)
    im2 = ax2.imshow(vid_arr[:,:,0,1], cmap=cmap, animated=True)
    im3 = ax3.imshow(vid_arr[:,:,0,2], cmap=cmap, animated=True)
    im4 = ax4.imshow(vid_arr[:,:,0,3], cmap=cmap, animated=True)
    im5 = ax5.imshow(vid_arr[:,:,0,4], cmap=cmap, animated=True)
    plt.close()
    
    for i, ax in enumerate([ax1, ax2, ax3, ax4, ax5]):
        ax.set(title='Scale' + str(i))
        ax.axis('off')

    # update
    def animate(f):
        im1.set_data(vid_arr[:,:,f,0])
        im2.set_data(vid_arr[:,:,f,1])
        im3.set_data(vid_arr[:,:,f,2])
        im4.set_data(vid_arr[:,:,f,3])
        im5.set_data(vid_arr[:,:,f,4])
        

    anim = animation.FuncAnimation(fig, animate, frames=vid_arr.shape[2], interval=50)
    plt.rcParams['animation.ffmpeg_path'] = ffmpeg_path
    return anim

## (A) Example input
As example, we will use one of the noisemasking stimuli from test case 1 of the paper with a medium grating frequency and a peak noise frequency of 9 cpd.

### 1. Parameters

In [ ]:
# Spatial resolution for simulations (pixels per degree).
ppd = 40

# Which test case to use as an example (options: case1, case2)
test_case = 'case1'

# Path to load natural images
input_path = params.data_path2

### 2. Create example stimulus
Select one example stimulus from either test case 1 (White stimulus with narrowband noise by Betz et al., 2015) or test case 2 (natural image from Contour Image Database by Grigorescu et al., 2003).

In [ ]:
# Test case 1: White stimulus in narrowband noise (Betz et al., 2015)
if test_case == 'case1':
    visual_extent = [-8., 8., -8., 8.]  # in deg
    
    # Grating frequency of White stimulus
    whitefreq = 'medium'

    # Noisefrequency of mask in cpd (Parameters taken from Betz2015)
    noisefreq = 9.
    
    # Create white stimulus in accordance to Betz2015
    white_stimulus = create_white_stimulus(whitefreq, ppd)
    white_size = white_stimulus.shape[0]

    # Add gray background that covers the desired visual extent
    back_size = visual_extent[1]*2.*ppd-white_size
    white_stimulus = np.pad(white_stimulus, int(back_size/2.), 'constant', constant_values=(1.))
    stim_size = white_stimulus.shape[0]

    # Add the desired noisemask to the stimulus
    stimulus = white_stimulus + create_noisemask(stim_size, noisefreq, ppd)

# Test case 2: Natural images (Grigorescu et al., 2003) with/without Gaussian white noise
elif test_case == 'case2':
    s_noise = 0.1  # Gaussian white noise added to input; either 0. or 0.1
    
    # Create a list with all image names of the Contour Image Database
    img_names = glob.glob(input_path + '*.pgm')
    img_names = [f[0:-4] for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]
    
    # Choose one stimulus:
    img_name = img_names[0]
    stimulus = cv2.imread(input_path + img_name + '.pgm')
    stimulus = cv2.cvtColor(stimulus, cv2.COLOR_BGR2GRAY) / 255.
    
    # Calculate image size in visual degree for the simulations
    stim_size = stimulus.shape[0]
    im_size_h = stimulus.shape[0] / ppd
    im_size_w = stimulus.shape[1] / ppd

    # Visual extent for the input stimuli:
    visual_extent = [-im_size_h/2, im_size_h/2, -im_size_w/2, im_size_w/2]  # in deg
    
    # Add Gaussian white noise and crop values larger than 1 and smaller than 0
    noise = np.random.normal(0., s_noise, [stim_size, stim_size])
    stimulus = stimulus + noise
    stimulus[stimulus <= 0.] = 0.
    stimulus[stimulus >= 1.] = 1.

### 3. Plot stimulus

In [ ]:
# Plotting
plt.figure(figsize=(6, 6))
plt.imshow(stimulus, cmap='gray', extent=visual_extent)
plt.colorbar()
plt.title('Example stimulus')
plt.xlabel('deg')
plt.ylabel('deg')
plt.show()

## (B) Active sampling / Applying drift

### 1. Parameters

In [ ]:
# Diffusion coefficient in deg**2/s (controls drift lengths)
D = 20. / (60.**2.)

# Total simulated fixation time in s
T = 0.2

# Drift sampling frequency in Hz / Temporal resolution
pps = 100.

### 2. Create drift trajectory

In [ ]:
# Create drift trajectories in px (continuous and int):
driftc_, drift = create_drift(T, pps, ppd, D)

In [ ]:
# Plotting
plt.figure()
plt.plot(driftc_[0,:] / ppd * 60., driftc_[1,:] / ppd * 60., 'o-', label='continuous')
plt.plot(drift[0,:] / ppd * 60., drift[1,:] / ppd * 60., 'o-', label='full pixels')
plt.title('Exemplary drift path')
plt.xlabel('x [arcmin]'), plt.ylabel('y [arcmin]'), plt.legend()
plt.show()

### 3. Apply drift to the input stimulus

In [ ]:
# Apply drift to stimulus to create dynamic video:
stimulus_video = apply_drift(stimulus, drift, back_lum=stimulus.mean())

In [ ]:
# Display actively-sampled input video:
anim = show_video(stimulus_video)
HTML(anim.to_html5_video())

## (C) Spatiotemporal filtering

### 1. Define spatial filters at multiple scales

In [ ]:
# Parameters:
# Number of DoG filters in multiscale filter bank:
n_filters = 5

# Sigma of largest DoG filter of the bank (deg):
smax = 0.256

# Define sigmas of all DoG filters in octave intervals
sigmas = octave_intervals(n_filters) * smax

# Define spatial frequency axis
nX = stim_size
fs = np.fft.fftshift(np.fft.fftfreq(nX, d=1./ppd))
fs_extent = (fs[0], fs[-1], fs[0], fs[-1])

# Create a list with all dog filters:
dogs_fft = []
for i in range(n_filters):
    # Create DoG filters in the frequency domain
    fx, fy = np.meshgrid(fs, fs)
    dog = create_dog(fx, fy, sigmas[i], 2.*sigmas[i])
    dogs_fft.append(dog)

In [ ]:
# Plot all filters in the frequency domain:
fig, axes = plt.subplots(1, n_filters, figsize=(18, 6), sharex=True, sharey=True)
fig.subplots_adjust(wspace=0.001)

for i in range(n_filters):
    axes[i].imshow(dogs_fft[i], cmap='coolwarm', extent=fs_extent)
    axes[i].set_title('Spatial scale: ' + str(i))
    axes[i].set_xlabel('cpd')
    axes[0].set_ylabel('cpd')

### 2. Define temporal bandpass filter

In [ ]:
# Define temporal frequency axis:
nT = int(T*pps + 1)
ft = np.fft.fftshift(np.fft.fftfreq(nT, d=1./pps))

# Create temporal filter:
tfilter = create_tfilt(ft)

In [ ]:
# Plot temporal filter
plt.figure(figsize=(8, 4))
plt.plot(ft, tfilter, 'o-')
plt.ylabel('Sensitivity')
plt.xlabel('tf (Hz)')
plt.show()

### 3. Perform spatiotemporal filtering

In [ ]:
# To filter the input in the freq-domain, we perform a 3d-fft of the actively-sampled input video
gfft_3d = np.fft.fftshift(np.fft.fftn(stimulus_video))

# Expand the dims of both filters to 3d (x, y ,t):
dogs_3d = []
for i in range(n_filters):
    dogs_3d.append(np.expand_dims(dogs_fft[i], -1))

tfilter_3d = np.expand_dims(tfilter, (0, 1))

In [ ]:
# Perform spatiotemporal filtering in the frequency domain and ifft
# The output contains the spatiotemporally-filtered input video at each spatial scale
output_filt = np.zeros([nX, nX, nT, n_filters])
for i in range(n_filters):
    output = gfft_3d * dogs_3d[i] * tfilter_3d
    output = np.fft.ifftn(np.fft.ifftshift(output))
    output_filt[:, :, :, i] = np.real(output)

In [ ]:
# Plot activation of all filters over time (scale 0-4 -> hsf-lsf)
anim = show_filter_videos(output_filt, 'coolwarm')
HTML(anim.to_html5_video())

In [ ]:
# Also visualize the activation after full-wave rectification (see next step)
anim = show_filter_videos(output_filt**2., 'pink')
HTML(anim.to_html5_video())

## (D) Temporal integration

In [ ]:
# At each scale, we integrate information across the time via the squared mean
output_int = (output_filt**2.).mean(2)

In [ ]:
# Plot the temporally integrated outputs for each scale
fig, axes = plt.subplots(1, n_filters, figsize=(18, 6), sharex=True, sharey=True)
fig.subplots_adjust(wspace=0.001)

# Let's compute the min/max to demonstrate the relevance of normalization
omin, omax = output_int.min(), output_int.max()

for i in range(n_filters):
    axes[i].imshow(output_int[:, :, i], cmap='pink', extent=visual_extent, vmin=omin, vmax=omax)
    axes[i].set_title('Spatial scale: ' + str(i))
    axes[i].set_xlabel('deg')
    axes[0].set_ylabel('deg')

## (E) Within-scale normalization

In [ ]:
# Remove borders before normalization to avoid border-effects
rb = 0.5
output_rem = remove_borders(output_int, int(rb*ppd))

# Normalize within each scale
output_norm = output_rem / np.expand_dims(output_rem.mean(axis=(0, 1)), (0, 1))

In [ ]:
# Plot the normalized outputs for each scale
fig, axes = plt.subplots(1, n_filters, figsize=(18, 6), sharex=True, sharey=True)
fig.subplots_adjust(wspace=0.001)

# Let's compute the min/max to demonstrate the relevance of normalization
omin, omax = output_norm.min(), output_norm.max()

for i in range(n_filters):
    axes[i].imshow(output_norm[:, :, i], cmap='pink', extent=visual_extent, vmin=omin, vmax=omax)
    axes[i].set_title('Spatial scale: ' + str(i))
    axes[i].set_xlabel('deg')
    axes[0].set_ylabel('deg')

## (F) Across-scale integration / Sum

In [ ]:
# Sum model responses across spatial scales:
model_output = np.sum(output_norm, 2)

# Adapted visual extent after border removal:
vextent_adapt = [visual_extent[0]+rb, visual_extent[1]-rb, visual_extent[2]+rb, visual_extent[3]-rb]

In [ ]:
# Plot the final model output
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.imshow(stimulus, cmap='gray', extent=visual_extent)
plt.colorbar()
plt.title('Example input')
plt.xlabel('deg')
plt.ylabel('deg')

plt.subplot(1, 2, 2)
plt.imshow(model_output, cmap='pink', extent=vextent_adapt)
plt.colorbar()
plt.title('Model output')
plt.xlabel('deg')
plt.ylabel('deg')
plt.show()

## (G) Evaluation

In [ ]:
# Create ground truth template with certain edge thickness (in deg)
edge_thickness = int(0.1*ppd)

if test_case == 'case1':
    # Create ground truth edge map
    gt_template = create_white_gt(stim_size, back_size, whitefreq, ppd, edge_thickness)

elif test_case == 'case2':
    # Import human-drawn ground truth and normalize between 0 (no contour) to 1 (contour)
    gt_template = cv2.imread(input_path + 'gt/' + img_name + '_gt_binary.pgm')
    gt_template = cv2.cvtColor(gt_template, cv2.COLOR_BGR2GRAY) / 255.
    gt_template = np.abs(gt_template-1.)
    
    # Thicken the edge signals to make them more comparable with our output
    gt_template = thicken_edges(gt_template, edge_thickness)

# Remove borders
gt_template = remove_borders(gt_template, int(rb*ppd))

In [ ]:
# Normalize model output (optional)
model_output = model_output / np.expand_dims(model_output.max(axis=(0, 1)), (0, 1))

# Quantify edge detection performance by aligning and correlating
# the model output and the ground truth
edge_corrs = quantify_edges(model_output, gt_template)

In [ ]:
# Plot model output and ground truth
plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.imshow(stimulus, cmap='gray', extent=visual_extent)
plt.colorbar()
plt.title('Input')
plt.xlabel('deg')
plt.ylabel('deg')

plt.subplot(1, 3, 2)
plt.imshow(gt_template, cmap='pink', extent=vextent_adapt)
plt.colorbar()
plt.title('Ground truth')
plt.xlabel('deg')
plt.ylabel('deg')

plt.subplot(1, 3, 3)
plt.imshow(model_output, cmap='pink', extent=vextent_adapt)
plt.colorbar()
plt.title('Model output')
plt.xlabel('deg')
plt.ylabel('deg')
plt.show()

# Print model performance, i.e. Pearson correlation with ground truth template
print('Model performance: ', edge_corrs)